# Supply Trend on fixture date

## Voyages Data & HTL joint Use Case

The following Use Case serves the need of monitoring the supply trend when a fixture is closed. We are going to use the voyages data api in order to retrieve the voyages of interest. Then, using the voyages data, we are going to query the HTL api and get the supply trend at the point in time that each vessel was fixed.  
For the sake of the example we are going to focus on voyages within 2021 having as commercial operator Vitol.

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
pip install signal-ocean

In [2]:
import pandas as pd
from datetime import date, timedelta,time

# imports regarding HTL
from signal_ocean import PortAPI, VesselClassAPI,PortFilter,VesselClassFilter,Connection
from signal_ocean.historical_tonnage_list import VesselFilter, PushType, MarketDeployment, CommercialStatus, VesselSubclass
from signal_ocean.historical_tonnage_list import HistoricalTonnageListAPI
from signal_ocean.historical_tonnage_list import Column, IndexLevel

# imports regarding Voyages Data
from signal_ocean.voyages import VoyagesAPI

In [3]:
signal_ocean_api_key = '' #replace with your subscription key
connection = Connection(signal_ocean_api_key)

In [4]:
# instantiating connections for all necessary apis 
port_api = PortAPI(connection)
vessel_class_api = VesselClassAPI(connection)
voyages_api = VoyagesAPI(connection)
htl_api = HistoricalTonnageListAPI(connection)

### Getting the voyages

In [5]:
vessel_class = vessel_class_api.get_vessel_classes(VesselClassFilter(name_like='aframax'))[0]
vessel_class_id = vessel_class.id

start_date = date(2021,1,1)

In [6]:
voyages = voyages_api.get_voyages(vessel_class_id=vessel_class_id, date_from=start_date)
voyages_df = pd.DataFrame([v.__dict__ for v in voyages])

In [7]:
# we need the voyages for which we know the fixture date
com_op_voyages = voyages_df[(voyages_df.commercial_operator=='Vitol') & \
                           (voyages_df.fixture_date.notna())].copy()

In [8]:
com_op_voyages.tail()

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,vessel_name,...,laycan_to,fixture_status_id,fixture_status,fixture_date,fixture_is_coa,fixture_is_hold,is_implied_by_ais,has_manual_entries,ballast_distance,laden_distance
4495,9821691,23,1,86,1,1831,False,"(VoyageEvent(id='I95DDFBSED81E9100', port_id=3...",I95DDFBVED81E9100,Elandra Bay,...,2021-05-28 00:00:00+00:00,1.0,FullyFixed,2021-05-11 10:01:51+00:00,False,False,None,None,611.80,NaN
4496,9821706,24,1,86,1,1831,False,"(VoyageEvent(id='I95DE0ASED7805D00', port_id=3...",I95DE0AVED7805D00,Elandra Sound,...,2021-01-11 19:55:25+00:00,1.0,FullyFixed,2021-01-05 14:26:27+00:00,False,False,None,None,549.77,5311.35
4498,9821706,26,1,86,1,1831,False,"(VoyageEvent(id='I95DE0ASED7C24800', port_id=1...",I95DE0AVED7C24800,Elandra Sound,...,2021-02-23 19:56:24+00:00,1.0,FullyFixed,2021-02-15 12:23:18+00:00,True,False,None,None,466.73,554.81
4501,9821706,29,1,86,1,1831,False,"(VoyageEvent(id='I95DE0ASED8116200', port_id=3...",I95DE0AVED8116200,Elandra Sound,...,2021-05-12 00:00:00+00:00,1.0,FullyFixed,2021-04-27 09:44:15+00:00,False,False,None,True,1691.13,1922.08
4554,9829899,20,1,86,1,1831,False,"(VoyageEvent(id='I95FE0BSED7CF7700', port_id=3...",I95FE0BVED7CF7700,Navig8 Pride Lhj,...,2021-03-26 23:59:00+00:00,1.0,FullyFixed,2021-03-12 02:44:42+00:00,False,False,None,None,2605.56,2874.46


We need the information regarding the first load of the voyage. It will be the basis port for the htl query.

In [9]:
def get_voyage_first_load_port(voyage_events):
    return next((e.port_name for e in voyage_events or [] if e.purpose=='Load'), None)

com_op_voyages['first_load_port'] = com_op_voyages['events'].apply(get_voyage_first_load_port)

The htl_query function is going to call the HTL api for each voyage, using as parameters its attributes.  

In [10]:
def htl_query(**params):
    port = port_api.get_ports(PortFilter(name_like=params['port_name']))[0]
    vessel_class = vessel_class_api.get_vessel_classes(VesselClassFilter(name_like=params['vessel_class']))[0]
    date_of_interest = params['fixture_date']
    laycanEndInDays = (params['laycan_end'] - date_of_interest).days
    
    vessel_filter = VesselFilter(
        push_types=[PushType.PUSHED, PushType.PUSHED_POSS],
        market_deployments=[MarketDeployment.RELET, MarketDeployment.SPOT],
        commercial_statuses=[CommercialStatus.AVAILABLE, CommercialStatus.CANCELLED, CommercialStatus.FAILED],
        vessel_subclass=VesselSubclass.DIRTY,
        latest_ais_since=5
    )
    
    htl_for_supply_trend = htl_api.get_historical_tonnage_list(
        port,
        vessel_class,
        laycanEndInDays,
        start_date = date_of_interest- timedelta(days=2),
        end_date = date_of_interest,
        vessel_filter=vessel_filter,
        time=time(hour=6)
    )
    supply_trend_data_frame = htl_for_supply_trend.to_data_frame()
    supply_trend = supply_trend_data_frame.groupby(IndexLevel.DATE, sort=True).size().to_dict()
    
    return supply_trend
    

In [14]:
print(f'# of calls to HTL api: {com_op_voyages.shape[0]}')

# of calls to HTL api: 43


In [11]:
com_op_voyages['supply_trend'] = com_op_voyages\
                                .apply(lambda row:htl_query(port_name = row['first_load_port'],
                                                            vessel_class = row['vessel_class'],
                                                            fixture_date = row['fixture_date'].date(),
                                                            laycan_end = row['laycan_to'].date()),axis=1)

In [12]:
com_op_voyages[['1d before','2d before','3d before']] = com_op_voyages['supply_trend']\
                                                       .apply(lambda x: list(x.values())).apply(pd.Series).values

In [13]:
com_op_voyages[['vessel_name','voyage_number','fixture_date','1d before','2d before','3d before']].head(10)

,vessel_name,voyage_number,fixture_date,1d before,2d before,3d before
442,Neverland,154,2021-02-19 08:27:33.390000+00:00,28,19,20
443,Neverland,155,2021-03-04 18:29:03+00:00,76,76,77
445,Neverland,157,2021-04-12 06:53:00.850000+00:00,1,2,2
446,Neverland,158,2021-04-19 10:36:52+00:00,33,39,48
448,Neverland,160,2021-05-10 21:38:06+00:00,219,216,205
1479,Kronviken,247,2021-01-14 18:10:15+00:00,66,58,57
1481,Kronviken,249,2021-02-26 09:51:48+00:00,68,82,74
1484,Solviken,234,2021-01-18 15:55:31.557000+00:00,16,20,22
1485,Solviken,235,2021-02-15 12:27:29+00:00,37,43,46
1486,Solviken,236,2021-02-26 17:01:39+00:00,37,45,40
